In [ ]:
#importing libraries
import html2text
from bs4 import BeautifulSoup
from selenium import webdriver 
import pandas as pd 
import requests
from lxml import html
from selenium.webdriver.common.by import By
import numpy as np
import itertools
from itertools import zip_longest
import time 

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36'}

driver = webdriver.Chrome()

#Getting all transaction hashes - scrape main address page 
page = driver.get('https://binance.mintscan.io/account/')
time.sleep(10)
subpage = driver.page_source
subpagesoup = BeautifulSoup(subpage, 'html.parser')
    
urls = []
tables = subpagesoup.find_all('table')
for table in tables: 
    #scrape by href to get all weblinks with txs. not possible to get txid by text due to text truncation in java
    data = table.find_all(href = True)
    for links in data:
        if '/txs' in links['href']: 
            #create list of tx urls 
            urls.append('https://binance.mintscan.io' + links['href'])
txsum =[]
for url in urls: 
    page = driver.get(url)
    time.sleep(10)
    subpage = driver.page_source
    subpagesoup2 = BeautifulSoup(subpage, 'html.parser')
    #scrape by tage 'li', finding pattern in tx url pages
    data = subpagesoup2.find_all('li')
    TxHash = data[1].text
    Status = data[3].text
    date = data[7].text
    Receiveraddress = data[9].text
    Value = data[11].text
    Sender = data[13].text
    txsum.append([TxHash, Status, date, Receiveraddress, Sender, Value])
    print(url)
export = pd.DataFrame(txsum)

#formatting dates 
for index, date in enumerate(export[2]):
    date = date.replace('/','')
    export[2][index] = date
export.to_csv()


